In [2]:
from temporora import *

In [ ]:
# Assuming metrics is defined before this loop
for dataset in datasets:
    print(f"Processing dataset: {dataset}")
    args.sub_dataset = dataset  # Set current dataset

    train_loader, valid_loader, test_loader, test_loader_last, \
        num_test_windows, train_visualize, engine_id = get_dataloader(
            dir_path=args.dataset_root,
            sub_dataset=args.sub_dataset,
            max_rul=args.max_rul,
            seq_length=args.sequence_len,
            batch_size=args.batch_size,
            use_exponential_smoothing=args.use_exponential_smoothing,
            smooth_rate=args.smooth_rate)

    # Fit linear regression model
    linear = LinearRegression()
    for i, (x, y) in enumerate(train_loader):
        x = x.view(-1, args.sequence_len * args.feature_num).numpy()
        y = y.numpy()
        linear.fit(x, y)

    # Validate and Test
    valid_predictions, valid_actuals = [], []
    test_predictions, test_actuals = [], []

    # Validation set
    for i, (x, y) in enumerate(valid_loader):
        x = x.view(-1, args.sequence_len * args.feature_num).numpy()
        y = y.numpy()
        preds = linear.predict(x)

        valid_predictions.extend(preds)
        valid_actuals.extend(y)

    # Test set
    for i, (x, y) in enumerate(test_loader):
        x = x.view(-1, args.sequence_len * args.feature_num).numpy()
        y = y.numpy()
        preds = linear.predict(x)

        test_predictions.extend(preds)
        test_actuals.extend(y)

    # Post-process predictions
    test_predictions = np.array(test_predictions)
    test_actuals = np.array(test_actuals)
    valid_predictions = np.array(valid_predictions)
    valid_actuals = np.array(valid_actuals)

    max_rul = max(test_actuals.max(), valid_actuals.max())
    test_predictions = np.clip(test_predictions, 0, max_rul)
    valid_predictions = np.clip(valid_predictions, 0, max_rul)
    test_predictions = test_predictions * args.max_rul
    test_actuals = test_actuals * args.max_rul

    # Compute Metrics
    valid_mse = mean_squared_error(valid_actuals, valid_predictions)
    valid_rmse = np.sqrt(valid_mse)
    valid_r2 = r2_score(valid_actuals, valid_predictions)

    test_mse = mean_squared_error(test_actuals, test_predictions)
    test_rmse = np.sqrt(test_mse)
    test_r2 = r2_score(test_actuals, test_predictions)

    metrics[dataset] = {
        "Validation MSE": valid_mse,
        "Validation RMSE": valid_rmse,
        "Validation R2": valid_r2,
        "Test MSE": test_mse,
        "Test RMSE": test_rmse,
        "Test R2": test_r2
    }

    print(f"Dataset {dataset} Metrics:")
    print(f"Validation MSE: {valid_mse:.4f}, RMSE: {valid_rmse:.4f}, R2: {valid_r2:.4f}")
    print(f"Test MSE: {test_mse:.4f}, RMSE: {test_rmse:.4f}, R2: {test_r2:.4f}")

    # Inline Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(test_actuals, label='Actual RUL', color='blue', linewidth=2)
    plt.plot(test_predictions, label='Predicted RUL', color='orange', linestyle='--', linewidth=2)
    plt.xlabel('Window')
    plt.ylabel('RUL')
    plt.title(f'Linear Regression Model - {dataset}')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()  # Inline visualization during the loop


Processing dataset: FD001


C:\Users\Piyush\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(1)